# Import modules

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import torchvision
from torchvision import datasets, models
from torchvision import transforms

import numpy as np

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import matplotlib.pyplot as plt

# Data Processing

In [3]:
zipurl = 'https://pytorch.tips/bee-zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('./data')

Note:
- Data augmentations to make models more robust to variations in size and position
    - RandomResizedCrop to 224x224 (standard input for many pretrained CNNs)
    - RandomHorizontalFlip (eg. if cat facing left or right)
- [ToTensor() docs](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html) also does
    - scale pixel values from range [0,255] to [0.0,1.0]
    - rearranges dimensions from (height, width, channels) to (channels,height,width) - format expected by PyTorch
- Normalize 
    - applies $\frac{(channel-mean)}{std}$ for each channel


Note:
- data augment only for training, **NOT** for validation

In [4]:
train_transforms = transforms.Compose([     # Chain multiple transformations
    transforms.RandomResizedCrop(224),      # Randomly crops image and resize to 224x224 
    transforms.RandomHorizontalFlip(),      # Randomly flip image horizontally (left-right) with default probability 0.5
    transforms.ToTensor(),                  # Converts PIL image or numpy array to PyTorch tensor
    transforms.Normalize(
        [0.485, 0.456, 0.406],              # ImageNet means for 3 channels (Red, Green, Blue)
        [0.229, 0.224, 0.225]               # ImageNet std dev for each channel
    )
])

In [6]:
val_transforms = transforms.Compose([       # Chain multiple transformations
    transforms.Resize(256),                 # Resize to 256x256
    transforms.CenterCrop(224),             # Crop in center 224x224
    transforms.ToTensor(),                  # Converts PIL image or numpy array to PyTorch tensor
    transforms.Normalize(
        [0.485, 0.456, 0.406],              # ImageNet means for 3 channels (Red, Green, Blue)
        [0.229, 0.224, 0.225]               # ImageNet std dev for each channel
    )
])

In [7]:
train_dataset = datasets.ImageFolder(
    root='data/hymenoptera_data/train',
    transform=train_transforms
)

In [8]:
val_dataset = datasets.ImageFolder(
    root='data/hymenoptera_data/val',
    transform=val_transforms
)

In [10]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,   # batch of 4 at time
    shuffle=True,   # shuffle
    num_workers=4   # configure for 4 CPU processors in parallel
)

In [11]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=4,   # batch of 4 at time
    shuffle=True,   # shuffle
    num_workers=4   # configure for 4 CPU processors in parallel
)

# Model Design

ResetNet18 model 
- pretrained with ImageNet data 
- designed to detect 1000 classes
    - in our case we only need 2 classes (bees and ants)
    - modify final layer to detect 2 classes instead of 1000

In [ ]:
# Downloads model to ~/.cache/torch/hub/checkpoints/resnet18-xxx.pth
model = models.resnet18(pretrained=True)

/Users/jasonlau/.pyenv/versions/3.11.11/envs/deeplearn/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jasonlau/.pyenv/versions/3.11.11/envs/deeplearn/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/jasonlau/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 30.2MB/s]


In [13]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
# read the number of featues before final layer
num_ftrs = model.fc.in_features
# change final layer by setting directly to fully connected layer with 2 outputs
model.fc = nn.Linear(num_ftrs, 2)

In [16]:
model.fc

Linear(in_features=512, out_features=2, bias=True)

going to 
- use the pretrained model as starting point
- fine-tune its parameters with new data
    - since we replaced the final linear layer, it's parameters are now randomly initialized

# Training and Validation

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
model = model.to(device)

In [20]:
criterion = nn.CrossEntropyLoss() # Define or loss function

In [ ]:
optimizer = optim.SGD(  # Define our optimizer algorithm
    model.parameters(),
    lr=0.001,
    momentum=0.9  
)

using a PyTorch scheduler to adjust the learning rate of our SGD optimizer after several epochs
- will help our NN adjust weights more precisely as training goes on

In [22]:
exp_lr_scheduler = StepLR( # Use a Learning Rate scheduler
    optimizer,
    step_size=7,
    gamma=0.1       
)

In [ ]:
num_epochs=25

for epoch in range(num_epochs):
    model.train()                       # Training loop
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = models(inputs)

        _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()/inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)/inputs.size(0)
    
    exp_lr_scheduler.step()         # Schedule the learning rate for next epoch of training

    train_epoch_loss = running_loss / len(train_loader)
    train_epoch_acc = running_corrects / len(train_loader)

    model.eval()                    # Validation loop
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)

        loss = criterion(outputs, labels)     

        running_loss += loss.item()/inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)/inputs.size(0)

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = running_corrects.double() / len(val_loader)

    print(f'Train: Loss:{train_epoch_loss:.4f} Acc:{train_epoch_acc:4f} Val: Loss:{epoch_loss:.4f} Acc:{epoch_acc:.4f}')

# Testing and Deployment

In [ ]:
def imshow(inp, title=None):                    # define a new function to plot images from our tensor images
    inp = inp.numpy().transpose((1,2,0))        # switch from C x H x W to H x W x C
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean                      # undo the normalizations during transforms to properly view images
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)    

In [ ]:
inputs, classes = next(iter(val_loader))            # Grab a batch of images from our validation dataset
out = torchvision.utils.make_grid(inputs)
class_names = val_dataset.classes

outputs = model(inputs.to(device))                  # Perform classification using our fine-tuned ResNet18
_, preds = torch.max(outputs, 1)                    # Take the "winning" class

imshow(out, title=[class_names[x] for x in preds])  # Display the input images and their predicted classes

In [ ]:
torch.save(model.state_dict(), "./resnet18.pt")